# Burgers' equation

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import seaborn as sns
sns.set_style('white',{'legend.frameon':'True'});
import matplotlib as mpl
mpl.rcParams['font.size'] = 8
figsize =(8,4)
mpl.rcParams['figure.figsize'] = figsize
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact
from ipywidgets import widgets, FloatSlider
from utils import riemann_tools
from exact_solvers import burgers
from exact_solvers import burgers_demos
from io import StringIO

In this chapter, we study another scalar nonlinear conservation law: Burgers' equation.  Burgers' equation models momentum transport in a fluid of uniform density and pressure, and it is the simplest equation that captures some key features of gas dynamics. We will employ the usual interactive approaches as in other chapters. 

If you wish to examine the Python code for this chapter, please see:
 - [exact_solvers/burgers.py](exact_solvers/burgers.py)
 - [exact_solvers/burgers_demos.py](exact_solvers/burgers_demos.py)

Burgers' equation has been used extensively for developing both theory and numerical methods, and it will allow us to further explore the Riemann problem for nonlinear conservation laws. The equation is given by
\begin{align}
q_t + \left(\frac{1}{2}q^2\right)_x = 0.
\label{burgers0}
\end{align}
It is a scalar conservation law with the nonlinear flux term $f(q)=q^2/2$. The quasilinear form is obtained by applying the chain rule to differentiate the flux term:  
\begin{align*}
q_t + qq_x = 0.
\end{align*}  
This equation looks very similar to an advection equation with the exception that the advection speed is given by the value $q$. This is similar to the traffic flow equation but with a different flux. The nonlinearity in the Burgers flux term is essentially the same as in the momentum conservation equation in fluid dynamics, so studying its dynamics provides a guideline to understand more complex fluid dynamics.

## Shock formation

We can begin by assuming Burgers' equation describes the momentum $q$ of some fluid and consider an initial condition given by a bump. As the advection speed is given by $q$ itself, the peak of the wave propagates faster than the bottom. In the next figure, we show a solution at two different times to illustrate this behavior.

![Fig. 5.1: Wave breaking for Burgers' equation](./figures/burgers_bump.png)

Notice that at first $q$ remains single-valued for every $x$. However, after some time the crest of the wave overtakes the leading edge. After this time, we obtain a triple-valued solution for certain values of $x$, which is unphysical since momentum is single-valued. The first time this overtaking happens is referred to as the *breaking time* -- a reference to waves breaking on the beach. It is also the point where the conservation law, in its differential form, breaks down and where the characteristics cross each other for the first time.  As we learned in [Traffic_flow.ipynb](Traffic_flow.ipynb), when characteristics cross, a shock wave forms. Mathematically, imposing a shock will avoid the problem of the solution being multivalued.  Where should the shock be located?

If we replace part of the multivalued solution interval with a shock, some mass will be removed (area A1 in the figure below) and some mass will be added (area A2 in the figure below).  In order to maintain conservation of the integral of $q$, the shock must be placed so that these two areas are equal:

![Fig. 5.2: The equal-area rule](./figures/burgers_equal_area.png)

This geometric reasoning provides a nice intuition for the shock location, but is cumbersome in practice.  To determine the path of the shock we can again use the Rankine-Hugoniot jump condition, derived in [Traffic_flow.ipynb](Traffic_flow.ipynb).  Since the flux for Burgers' equation is $f(q) = q^2/2$, we have  
\begin{align*}
s(q_r-q_l) & = f(q_r) - f(q_l)\\
           & = \frac{1}{2} (q_r^2 - q_l^2) \\
\Rightarrow \ \ \ \ s &= \frac{1}{2}(q_l + q_r).
\end{align*}  
Here $q_l, q_r$ denote the solution values just to the left and right of the shock.  In general (as in the image above) these states will not be constant and the speed of a shock will change in time.

### Shock solution
For Burgers' equation, as for the traffic model (or any scalar hyperbolic conservation law with a convex flux function), the solution of the Riemann problem will always consist of one wave, which may be either a shock or a rarefaction.  The analysis above already yields the solution to the Riemann problem in the case that the resulting wave is a shock.  The entropy condition, as we will explain below, determine a shock will form when $f'(q_l) > f'(q_r)$; i.e. when $q_l>q_r$.  In this case, the solution is 

\begin{align*}
q(x,t) = 
\begin{cases}
q_l \quad \text{if} \ \ x/t<s \\
q_r \quad \text{if} \ \ x/t>s.
\end{cases}
\end{align*}  
We will now plot the solution of the Burgers' equation when shocks are formed, i.e. when $q_l>q_r$. We first define an interactive plotting function for the exact solver and then we choose the initial conditions and plot the solution. 

In [ ]:
interact(burgers_demos.shock(), t=widgets.FloatSlider(value=0.0,min=0,max=1.0),
                    which_char=widgets.Checkbox(value=True,description='Show characteristics'));

## Rarefaction wave
In the previous figure with the hump as the initial condition, we observed that a shock formed on the right side of the hump. However, on the left side, the characteristics spread out and will never cross. This part of the solution is therefore a rarefaction wave. This is the kind of behavior we will observe in the solution of the Riemann problem when $q_l<q_r$.

In the next figure, we consider such a Riemann problem. As time evolves, a rarefaction forms following the advection speed $q$ given by the quasi-linear equation. Therefore, for time $t$, the solution must propagate a distance $x=qt$, so the solution along the rarefaction is then $q = x/t$. As $q_l<q_r$, the smallest and largest displacements are given by $q_l t$ and $q_r t$, respectively. 

![Burgers_rarefaction.](./figures/burgers_rar.png)

The full rarefaction solution for the Burgers' Riemann problem is then simply given by 
\begin{align*}
q(x,t) = 
\begin{cases}
q_l, \quad \text{for} \ \ x<q_l t \\
x/t, \quad \text{for} \ \ q_l t \le x \le q_r t \\
q_r, \quad \text{for} \ \ x>q_r t.
\end{cases}
\end{align*}

As we will see in the next chapter, the rarefaction solution is always a self-similar solution. This means that it can be expressed as a function of the ratio between position and time $q(x,t) = \tilde{q}(x/t)$, so it remains the same when rescaling both $x$ and $t$ by the same factor. This is a consequence of $q$ being the same along the characteristic in the rarefaction fan. In the Burgers' equation, the form of the rarefaction is particularly simple since the advection speed is simply $q$.

### Rarefaction solution
In the figure below, we plot solutions of the Riemann problem with $q_l<q_r$. In these cases, we will always observe a rarefaction.

In [ ]:
interact(burgers_demos.rarefaction(), t=widgets.FloatSlider(value=0.0,min=0,max=1.0),
                    which_char=widgets.Checkbox(value=True,description='Show characteristics'));

## Weak solutions

As we mentioned before, the differential form of the equation breaks down in the presence of shocks/discontinuities. However, the integral form of the conservation law remains valid. We will derive a specific form of the integral conservation law that is particularly useful to prove mathematical results.
We first intergate the conservation law $q_t+f(q)_x=0$ from from $x=x_1$ to $x=x_2$ and $t=t_1$ to $t=t_2$

\begin{align}
\int_{t_1}^{t_2}\int_{x_1}^{x_2} [q_t+f(q)_x] dx dt = 0,
\label{eq:Burgersintclaw}
\end{align}
where $f(q)=q^2/2$ for Burgers' equation. However, this derivation is general and applies to all nonlinear scalar conservation laws. This equation can be simply rewritten in terms of a function $\phi(x,t)$ with compact support in $[x_1,x_2]\times[t_1,t_2]$ (1 in this region, zero elsewhere),

\begin{align*}
\int_{0}^{\infty}\int_{-\infty}^{\infty} [q_t+f(q)_x]\phi(x,t) dx dt = 0.
\end{align*}

Integrating by parts we obtain

\begin{align*}
\int_{0}^{\infty}\int_{-\infty}^{\infty} [q\phi_t+f(q)\phi_x] dx dt = -\int_{-\infty}^{\infty}q(x,0)\phi(x,0)dx,
\end{align*}
where now the derivatives are on $\phi(x,t)$ and not on $q$ or $f(q)$, so the equation still makes sense for discontinuous $q$. The function $q(x,t)$ is a weak solution of the conservation law if this equation holds for all functions $\phi(x,t)$ that are continuously differentiable and with compact support (bump functions). Note this rules out the original interval chosen in Eq. \ref{eq:Burgersintclaw} since its $\phi$ would not be smooth. However, we can approximate this function arbitrarily well by a smooth function. Note any weak solution is a solution of any form of the integral conservation law and vice versa.

The concept of weak solutions is helpful to do mathematical proofs and manipulations. Actually, all the solutions previously derived in this chapter are weak solutions of Burgers' equation. However, there is a caveat; weak solutions are not always unique.  

### An unphysical weak solution

We previously mentioned that for Burgers' equation, when $q_l<q_r$, we expect a rarefaction to be the correct solution. This is because if we smeared out the initial data just slightly then there would be a smoothly varying characteristics emerging from each point $x$ and these characteristics would spread out.  However, for exactly discontinuous initial data, we could also assume the solution consists of a propagating discontinuity and then use the equation in its conservative form to derive the Rankine-Hugoniot conditions, as done in previous chapters and in <cite data-cite="fvmhp"><a href="riemann.html#fvmhp">(LeVeque, 2002)</a></cite>. This is mathematically correct and one obtains the same relation for the shock speed as before, $s = (q_l + q_r)/2$ , so the solution is simply

\begin{align*}
q(x,t) = 
\begin{cases}
q_l \quad \text{if} \ \ x/t<s \\
q_r \quad \text{if} \ \ x/t>s.
\end{cases}
\end{align*}

This unphysical solution is also referred to as expansion shock, and it is also a weak solution to the Burgers' equation. In the next figure, we plot this weak solution.

In [ ]:
interact(burgers_demos.unphysical(), t=widgets.FloatSlider(value=0.0,min=0,max=1.0),
                    which_char=widgets.Checkbox(value=True,description='Show characteristics'));

Note the speed of the characteristics with respect to the shock. The spreading characteristics clearly indicate that the correct solution should be a rarefaction. In order to be able to specify which of the weak solutions is physically correct, we need to derive a mathematical condition from our physical intuition gained from observing the behavior of the characteristics. This condition is referred as the entropy condition.

### The entropy condition

In the traffic flow chapter, we already briefly mentioned the entropy condition in a very intuitive context. More generally, the entropy condition determines which is the correct physical solution among all the possible weak solutions. In the case of a specific Riemann problem of a conservation law, the entropy condition will essentially allow us to easily determine if the physical solution should be a shock or a rarefaction. In the case of scalar conservation laws, the entropy condition is quite simple and can be formulated in terms of the flux function of the conservation law as follows: the solution of a scalar Riemann problem will consist of a shock only if

$$
f'(q_l) > f'(q_r).
$$
In other words, the solution is a shock only if the characteristics are going into the shock as time progresses. If the characteristics are spreading out/going out of the shock as in the last example, one would naturally expect a rarefaction to be the correct solution. In the case of Burgers' equation, the flux function is $f(q)=q^2/2$, so the correct solution is a shock only if

$$
q_l > q_r,
$$
which can be clearly observed in the interactive solution below. The name, "entropy condition", comes from gas dynamics, where the physical entropy must increase in the gas passing through a shock, according to the second law of thermodynamics.  A discontinuity that violates this is non-physical. A mathematical version of an "entropy function" can be defined for other conservation laws.  More discussion of this and several other formulations and more detailed explanations available in the literature <cite data-cite="fvmhp"><a href="riemann.html#fvmhp">(LeVeque, 2002)</a></cite>. In further chapters, we will also explore how this condition generalizes to non-scalar systems.

### Convexity

The flux $f(q) = q^2/2$ for Burgers' equation is "convex" in the sense that $f''(q) \neq 0$ for all values of $q$ (in fact $f''(q) \equiv 1$). This means that as $q$ varies between $q_l$ and $q_r$ the the characteristic speed $f'(q) = q$ is either monotonically increasing (if $q_l < q_r$) or monotomically decreasing (if $q_l > q_r$).  Because of this the solution is always either a single rarefacetion wave in the former case or a single shock wave in the latter case.

The flux $f(\rho) = \rho(1-\rho)$ from the LWR traffic flow model  is also "convex" in this sense since $f''(\rho) = -2$ for all $\rho$ and is never zero.  Since it is always negative, as we saw in [Traffic_flow.ipynb](Traffic_flow.ipynb), the correct Riemann solution is a shock if $\rho_l < \rho_r$, or a rarefaction wave if $\rho_l > \rho_r$, opposite to the situation in Burgers' equation, where $f''(q)$ is always positive.

As you might guess, the solution to the Riemann problem can be much more complicated if $f'(q)$ is not monotonically varying between $q_l$ and $q_r$, i.e. if $f''(q)$ changes sign.  In this case the Riemann solution can consist of multiple shock and rarefaction waves.  The nonconvex case is explored further in [Nonconvex_scalar.ipynb](Nonconvex_scalar.ipynb).

## Interactive solution and examples
We can now explore a few more examples that are representative of phenomena we will observe in more complicated systems. Before exploring more complicated examples, we first show a full interactive solution for the Riemann problem for Burgers' equation. The values of the initial conditions and the time can be modified interactively to observe how it affects the characteristic structure and the solution. 

In [ ]:
burgers.riemann_solution_interact()

### Example 1: Bump initial condition
In order to see how a shock forms from an initial bump initial condition, we can use "pyclaw" to produce an animated plot of the solution. We can clearly observe how the bump initial condition transitions into a shockwave solution. Note it takes some time to produce the animation since the code is actually solving the equation for all the desired frames.

In [ ]:
burgers_demos.bump_animation(numframes = 50)

### Example 3: Three state solution

When three piecewise constant values are given as initial condition, one can solve a Riemann problem across each discontinuity. For instance, consider the Burgers' equation with its initial condition given by

$$
\begin{align*}
q_0(x) = 
\begin{cases}
q_l \quad \text{if} \ \ x < -1 \\
q_m \quad \text{if} \ \ -1\le x \le 1 \\
q_r \quad \text{if} \ \ x > 1.
\end{cases}
\end{align*}  
$$

We can decompose this into two Riemann problems, one at $x=-1$ and another one at $x=1$. Each of these two Riemann problems will yield either a shock or a rarefaction. The interesting part is what happens when the generated shocks or rarefactions interact. Following the animation below, let us assume that both Riemann problems produce a right-propagating shock. However, the shock generated at $x=-1$ propagates faster, so it will eventually reach the shock originally generated at $x=1$. At the point in time when the shocks collide, one can simply restate the problem again as a Riemann problem and solve the whole problem analytically. We again use "pyclaw" to solve the problem and show an animated solution.

In [ ]:
burgers_demos.triplestate_animation(ql = 4.0, qm = 1.0, qr = 0.0, numframes = 50)

As one would expect, this can become more complicated when one or both of the waves generated are given by rarefactions. The animation below shows how the right-propagating shock takes over the rarefaction. Try modifying the values of $q_l$, $q_m$ and $q_r$ in order to see what other behavior can be observed. Note one can also solve this problem analytically and draw its corresponding characteristic curves.

In [ ]:
burgers_demos.triplestate_animation(ql = 4.0, qm = -0.5, qr = 0.5, numframes = 50)